# Annotation of TCR Sequences with gapped alignment
Here, I provide the scripts used to perform the heavy lifting on this project. The script `vdjdb_search` whatever you give it to the database, and `batch_vdjdb_align` will do a bunch of files in a smarter fashion. I also provide a few samples to test out. 


In [8]:
# Load the required packages for my script
suppressWarnings(suppressMessages(library(tidyverse, Biostrings)))

In [3]:
## Alignment Scripts ##
suppressMessages(require(Biostrings))

vdjdb_search <-
  function(query_list,
           seq_db,
           strand,
           retain_extras = FALSE,
           score = "base") {
    # Either use the alpha or the beta strand, depending on query sequences
    if (missing(strand)) {
      stop("Please specify an alpha or beta strand")
    } else if (strand == "alpha") {
      cdr3s <- subset(seq_db$cdr3.alpha, !is.na(seq_db$cdr3.alpha))
    } else if (strand == "beta") {
      cdr3s <- subset(seq_db$cdr3.beta, !is.na(seq_db$cdr3.beta))
    } else{
      stop("Please specify an alpha or beta strand")
    }
    
    # Make a progress bar object
    pb <- txtProgressBar(style = 3,
                         min = 0,
                         max = length(query_list))
    progress <- 1
    
    # Perform alignment of a query sequence against the whole VDJdb
    # We can use gap opening / extension penalties, or specify a BLOSUM matrix
    out <- lapply(query_list, function(query) {
      if (score == "base") {
        align <- pairwiseAlignment(cdr3s, query)
      } else if (score == "BLOSUM") {
        align <-
          pairwiseAlignment(cdr3s, query, substitutionMatrix = "BLOSUM50")
      } else if (score == "AB") {
        # This was from when I tried making a BLOSUM matrix from the Mirsky 2014 Study
        align <-
          pairwiseAlignment(cdr3s, query, substitutionMatrix = BLOSUMAB)
      } else
        stop("BLOSUM is the only valid parameter for score")
      
      # Select the sequence in the DB which aligns best
      best_match <- which.max(score(align))
      score <- score(align)[best_match]
      alignment <- aligned(align)[best_match] %>% as.vector
      n_mismatches <- nmismatch(align)[best_match]
      
      # Compile important results
      my_tibble <- tibble(
        "query" = query,
        "best_match" = cdr3s[best_match],
        "score" = score,
        "alignment" = alignment,
        "n_mismatches" = n_mismatches
      )
      
      # Update the progress bar
      setTxtProgressBar(pb, progress)
      progress <<- progress + 1
      
      # Select important things from the VDJdb file
      their_tibble <- seq_db %>%
        filter(cdr3.beta == cdr3s[best_match]) %>%
        select(antigen.species,
               mhc.a,
               antigen.epitope,
               antigen.gene,
               vdjdb.score) %>%  .[1,]
      
      cbind(my_tibble, their_tibble)
      
    })  %>% do.call("rbind", .) %>% as.tibble
    close(pb)
    out
  }

This script lets you align a bunch of TCR annotation tables at once

In [38]:
batch_vdjdb_align <- function(cdr3_table_list, 
                              seq_db,
                              strand,
                              retain_extras = FALSE,
                              score = "base")  {
  
  finished <- 1
  
  lapply(cdr3_table_list, function(x){
    
    # Should we get rid of ambiguous symbols, or delete all sequences containing ambiguous symbols?
    if (retain_extras == "clean_ambiguous") {
      message("Removing extra base symbols _ and * from input query")
      x %<>% mutate(cdr3aa = str_remove_all(cdr3aa, "_|//*"))
    } else if (retain_extras == "remove_ambiguous"){
      message("Not considering aa sequences derived from non mod 3 nt sequences")
      x <- x[!x$cdr3aa %>% str_detect("_|//*") ,]
      x <- x %>% mutate(freq = count / sum(count))
    } else message("Processing raw imput")
    
    num_to_align <- min(25, nrow(x), which(cumsum(x$freq) > .8)[1])
    result <-
      x$cdr3aa[1:num_to_align] %>% vdjdb_search(
        vdjdb_human,
        strand = strand,
        retain_extras = retain_extras,
        score = score
      )
    result <-
      cbind((x %>% select(count, freq))[1:nrow(result), ], result)
    
    message(paste("Finished", finished, "samples"))
    finished <<- finished + 1
    result
  })
}

Read in the VDJdb and the sample files

In [31]:
vdjdb_human <- suppressMessages(read_tsv("vdjdb_human_beta.tsv", ) )
head(vdjdb[,1:10])

cdr3.alpha,v.alpha,j.alpha,cdr3.beta,v.beta,d.beta,j.beta,species,mhc.a,mhc.b
NA,NA,NA,CASSSGQLTNTEAFF,TRBV9*01,NA,TRBJ1-1*01,HomoSapiens,HLA-A*02:01,B2M
NA,NA,NA,CASSASARPEQFF,TRBV9*01,NA,TRBJ2-1*01,HomoSapiens,HLA-A*02:01,B2M
NA,NA,NA,CASSSGLLTADEQFF,TRBV9*01,NA,TRBJ2-1*01,HomoSapiens,HLA-A*02:01,B2M
NA,NA,NA,CASSSGQVSNTGELFF,TRBV9*01,NA,TRBJ2-2*01,HomoSapiens,HLA-A*02:01,B2M
NA,NA,NA,CSARDRTGNGYTF,TRBV20-1*01,NA,TRBJ1-2*01,HomoSapiens,HLA-A*02:01,B2M
NA,NA,NA,CSARGDGQGDLLQETQYF,TRBV20-1*01,NA,TRBJ2-5*01,HomoSapiens,HLA-A*02:01,B2M


In [26]:
sample_files <- suppressMessages(lapply(list.files("sample_samples/", full.names = TRUE), read_tsv))
names(sample_files) <- tools::file_path_sans_ext(list.files("sample_samples/"))

sample_files$bcn1[1:20,]

count,freq,cdr3nt,cdr3aa,v,d,j,VEnd,DStart,DEnd,JStart
70,0.053272451,TGTGCCTGGGGCGGGGGGAACTACGAGCAGTACTTC,CAWGGGNYEQYF,TRBV30,TRBD2,TRBJ2-7,10,11,17,20
62,0.047184170,TGCAGTGCTAAGAACCGTGGACCAGGAAGTGCAATTTT,CSAKNR_TRKCNF,TRBV20-1,TRBD1,TRBJ1-3,9,22,25,33
37,0.028158295,TGTGCAGAAGTTCTCGAATCATTCTTT,CAEVLESFF,TRBV15,TRBD2,TRBJ1-1,4,12,14,21
35,0.026636225,TGTGCCCAGTCCTTCTCCAAGAACTTC,CAQSFSKNF,TRBV6-4,TRBD2,TRBJ2-5,5,7,11,22
33,0.025114155,TGTGCCAGCAGCCTCGGGGACCAGACCGGGGAGCTGTTTTTT,CASSLGDQTGELFF,TRBV5-6,TRBD1,TRBJ2-2,11,16,20,24
27,0.020547945,TGCAGCGGGATCAAGCTGTGTGCATTAGTACAAATGGTGCAGTTTTT,CSGIKLCA_STNGAVF,TRBV29-1,TRBD1,TRBJ1-4,6,15,18,42
22,0.016742770,TGTGCCAGTAGTCGCCGTACCGGGGAGCTGTTTTTT,CASSRRTGELFF,TRBV19,TRBD1,TRBJ2-2,11,13,15,18
21,0.015981735,TGCAGCGCCCCCGATATCACCTGTTCTTC,CSAPD_HLFF,TRBV29-1,TRBD2,TRBJ2-1,7,8,12,22
20,0.015220700,TGTGCTGCATATTGGCCCTGTCTTT,CAAY_GPVF,TRBV21-1,TRBD1,TRBJ1-1,4,15,19,20
19,0.014459665,TGTGCCAGCAGTTCCCTCGACGGGAATGAAAAACTGTTTTTT,CASSSLDGNEKLFF,TRBV12-4,TRBD2,TRBJ1-4,12,13,17,24


Running just five samples takes about a minute, so to run the full 191, we're looking at about an hour of runtime... Still not great. 

In [40]:
batch_vdjdb_align(cdr3_table_list = sample_files, seq_db = vdjdb_human, strand = "beta", retain_extras = "remove_ambiguous")


Not considering aa sequences derived from non mod 3 nt sequences


  |======================================================================| 100%


Finished 1 samples
Not considering aa sequences derived from non mod 3 nt sequences


  |======================================================================| 100%


Finished 2 samples
Not considering aa sequences derived from non mod 3 nt sequences


...

  |======================================================================| 100%


Finished 4 samples
Not considering aa sequences derived from non mod 3 nt sequences


  |======================================================================| 100%


Finished 5 samples


count,freq,query,best_match,score,alignment,n_mismatches,antigen.species,mhc.a,antigen.epitope,antigen.gene,vdjdb.score
70,0.067698259,CAWGGGNYEQYF,CASGGGLYEQYF,30.5586052,CASGGGLYEQYF,2,InfluenzaA,HLA-A*02,GILGFVFTL,M1,0
37,0.035783366,CAEVLESFF,CAWSEVHEQFF,-0.3524046,CAEVHEQFF,2,CMV,HLA-A*02,NLVPMVATV,p65,0
35,0.033849130,CAQSFSKNF,CASSFSKNTEAFF,2.1903057,CASSFSKNF,1,HIV-1,HLA-B*42:01,TPQDLNTML,p24,1
33,0.031914894,CASSLGDQTGELFF,CASSLGPNTGELFF,39.1659431,CASSLGPNTGELFF,2,CMV,HLA-A*02,NLVPMVATV,p65,0
22,0.021276596,CASSRRTGELFF,CASSRRTGELFF,51.6440239,CASSRRTGELFF,0,HIV-1,HLA-B*27:05,KRWIILGLNK,p24,0
19,0.018375242,CASSSLDGNEKLFF,CASSSRSGNEKLFF,39.1659431,CASSSRSGNEKLFF,2,InfluenzaA,HLA-A*02:01,GILGFVFTL,M1,0
15,0.014506770,CASSRGQRYTEAFF,CASSFGQRETEAFF,39.1659431,CASSFGQRETEAFF,2,HomoSapiens,HLA-A*02:01,KLMNIQQKL,AKAP13,0
14,0.013539652,CASSSPTRLETQYF,CASSSRDREETQYF,28.6232300,CASSSRDREETQYF,3,HIV-1,HLA-B*42,TPQDLNTML,p24,0
13,0.012572534,CASSSPRAPSTDTQYF,CASSSPASSTDTQYF,40.0123215,CASSSP-ASSTDTQYF,1,HCV,HLA-A*01:01,ATDALMTGY,NS3,1
13,0.012572534,CASSLLGGQETQYF,CASSLLAGQETQYF,49.7086525,CASSLLAGQETQYF,1,EBV,HLA-B*08:01,RAKFKQLL,BZLF1,1
